# LegacyHalos Coadds

The goal of this notebook is to generate LegacySurvey (*grzW1W2*) coadds of all the galaxies in the legacyhalos parent sample (*legacyhalos-upenn-parent.fits*).

This notebook must be run from the jupyter-dev.nersc.gov notebook server in order to access the LegacySurvey data.

We assume that the \$LEGACYHALOS_DIR directory exists (in $CSCRATCH) with all the latest parent catalogs.

### Imports

In [1]:
import os
import numpy as np
from astropy.io import fits

from legacypipe.survey import LegacySurveyData, wcs_for_brick, read_one_tim
from legacypipe.runbrick import stage_tims, _get_mod, rgbkwargs, rgbkwargs_resid
from legacypipe.survey import get_rgb, imsave_jpeg
from legacypipe.coadds import make_coadds
from astrometry.util.multiproc import multiproc
from astrometry.util.fits import fits_table
from legacypipe.catalog import read_fits_catalog

### Paths and filenames

In [2]:
legacyhalos_dir = os.path.join( os.getenv('SCRATCH'), 'legacyhalos' )
parentfile = os.path.join(legacyhalos_dir, 'legacyhalos-upenn-parent.fits')

In [3]:
coaddsdir = os.path.join(legacyhalos_dir, 'coadds')
if not os.path.exists(coaddsdir):
    os.makedirs(coaddsdir)

### Read the Tractor photometry from the parent catalog.

In [4]:
parent = fits_table(parentfile, ext='LSPHOT')
print('Read {} galaxies from {}'.format(len(parent), parentfile))

Converted brickname from |S8 to <U8
Converted type from |S4 to <U4
Converted wise_coadd_id from |S8 to <U8
Read 1562 galaxies from /global/cscratch1/sd/ioannis/legacyhalos/legacyhalos-upenn-parent.fits


In [5]:
print('Hack!')
parent = parent[0:11]

Hack!


### Generate coadds centered on each object in the sample.

In [6]:
survey = LegacySurveyData(cache_dir='/project/projectdirs/cosmo/data/legacysurvey/dr5')
mp = multiproc()

In [7]:
print('The cutout size should adapt with the size of the object!')
sz = 100

The cutout size should adapt with the size of the object!


In [8]:
bands = ['g', 'r', 'z']

In [9]:
for ii, gal in enumerate(parent):
    coaddprefix = os.path.join(coaddsdir, 'coadd-{0:05d}'.format(gal.objid))
    #coaddprefix = os.path.join(coaddsdir, 'coadd-{0:04d}'.format(ii))

    bbox = [gal.bx-sz, gal.bx+sz, gal.by-sz, gal.by+sz]
    P = stage_tims(brickname=gal.brickname, survey=survey, target_extent=bbox,
                   pixPsf=True, hybridPsf=True, depth_cut=False, mp=mp)
    print('Got', P.keys())

    tims = P['tims']
    targetwcs = P['targetwcs']
    H, W = targetwcs.shape

    # Read the full Tractor catalog for this brick.
    fn = survey.find_file('tractor', brick=gal.brickname)
    cat = fits_table(fn)
    print('Read {} sources from {}'.format(len(cat), fn))
    
    # Restrict to just the sources in our little box. 
    ok, xx, yy = targetwcs.radec2pixelxy(cat.ra, cat.dec)
    I = np.flatnonzero((xx > 0) * (xx < W) * (yy > 0) * (yy < H))
    cat.cut(I)
    print('Cut to {} sources within our box.'.format(len(cat)))
    
    # Remove the central galaxy.
    I = np.flatnonzero(cat.objid != gal.objid)
    cat.cut(I)
    print('Removed central galaxy with objid = {}'.format(gal.objid))
    
    print('Creating tractor sources...')
    srcs = read_fits_catalog(cat, fluxPrefix='')
    print('Sources:')
    for src in srcs:
        print(' ', src)

    print('Rendering model images...')
    mods = [_get_mod((tim, srcs)) for tim in tims]

    print('Producing coadds...')
    C = make_coadds(tims, bands, targetwcs, mods=mods, mp=mp)
    print('Coadds:', dir(C))

    coadd_list= [('image', C.coimgs,   rgbkwargs),
                 ('model', C.comods,   rgbkwargs),
                 ('resid', C.coresids, rgbkwargs_resid)]
    #C.coimgs, C.comods, C.coresids
    for name,ims,rgbkw in coadd_list:
        rgb = get_rgb(ims, bands, **rgbkw)
        kwa = {}
        #with survey.write_output(name + '-jpeg', brick=brickname) as out:
        #    imsave_jpeg(out.fn, rgb, origin='lower', **kwa)
        #    print('Wrote', out.fn)
        outfn = '{}-{}.jpg'.format(coaddprefix, name)
        print('Writing {}'.format(outfn))
        imsave_jpeg(outfn, rgb, origin='lower', **kwa)
        del rgb
        

Converted brickname from |S8 to <U8
Got brick: Wall: 0.67 s, CPU: 0.68 s
Got brick wcs: Wall: 0.67 s, CPU: 0.68 s
Got git version: Wall: 0.70 s, CPU: 0.70 s
Got FITS header: Wall: 0.71 s, CPU: 0.71 s
Reading CCDs from /global/cscratch1/sd/desiproc/dr5/survey-ccds-lp25.fits.gz
Converted image_filename from |S65 to <U65
Converted camera from |S7 to <U7
Converted ccdname from |S4 to <U4
Converted object from |S35 to <U35
Converted propid from |S10 to <U10
Converted filter from |S1 to <U1
Got 11718 CCDs
Reading CCDs from /global/cscratch1/sd/desiproc/dr5/survey-ccds-nondecals-dr5.fits.gz
Converted object from |S37 to <U37
Converted filter from |S1 to <U1
Converted date_obs from |S10 to <U10
Converted ut from |S15 to <U15
Converted ha from |S13 to <U13
Converted propid from |S10 to <U10
Converted ccdname from |S3 to <U3
Converted temp from |S32 to <U32
Converted camera from |S5 to <U5
Converted expid from |S12 to <U12
Converted image_filename from |S65 to <U65
Got 551580 CCDs
Reading CCDs f